# Ground-truth generation pipeline (In progress)

Pipeline to generate Lidar or GNSS ground-truth from the prism trajectories gathered by the RTS setup

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_function as theodo_f
import scripts.prediction_utils as prediction_u
import scripts.resection_functions as theodo_r
import time
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [ ]:
theodo_u = importlib.reload(theodo_u)

path = "../data/20221110/"

Tf = theodo_u.read_saved_tf(path+"list_tf/TF_list_static_cp.csv")
Tf_1 = Tf[0]
Tf_12 = Tf[1]
Tf_13 = Tf[2]

path_GNSS_file = path+"gps_data/"+"gpsmiddle.txt"
GNSS_raw_data = theodo_u.read_prediction_data_Linear_csv_file(path_GNSS_file)
time_delay = float(theodo_u.read_time_delay(path+"gps_data/delay_synchronization_GNSS_3.txt"))

GNSS_raw_data_corrected = []
for i in GNSS_raw_data:
    raw_data = np.array([i[0]+time_delay, i[1], i[2], i[3], 0, 0, 0, 1])
    GNSS_raw_data_corrected.append(raw_data)
GNSS_raw_data_corrected = np.array(GNSS_raw_data_corrected)

In [ ]:
# Path of the rosbags
file = [
        "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
       ]

# Parameters to select:
# 1. Apply filtering or not (Module 1)
# 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1)
# 5. Parameter tau_s (Module 2)

parameters = [
    [1,2,1,1,3],
]

# Path of output
output = [
        "../data/20220224/"
]

In [ ]:
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)
prediction_u = importlib.reload(prediction_u)

for param in parameters:
    print(param)

    if(param[0]==0):
        filtering = False
    if(param[0]==1):
        filtering = True
    thresold_d = param[1]                # tau_r [m/s]
    thresold_a = param[2]                # tau_a [deg/s]
    thresold_e = param[3]                # tau_e [deg/s]
    limit_time_interval = param[4]       # tau_s [s]
    size_interval = 2

    Mode = "L"       # Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library
    limit_search = limit_time_interval
    save = False

    save_index_1 = []
    save_index_2 = []
    save_index_3 = []

    for fname, opath in zip(file,output):
        if(not filtering):
            path_out = opath + "raw_prediction/"
        else:
            path_out = opath + "filtered_prediction/"

        if(filtering):
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1),len(t2),len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(fname)
            print(len(t1),len(t2),len(t3))

        start_time = time.time()

        # Put trajectories in same frame
        tp1 = Tf_1@tp1
        tp2 = Tf_12@tp2
        tp3 = Tf_13@tp3

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []
        Index_sensor = []

        for i in tqdm(list_trajectories_split):

            index_1 = np.array([i[0,0],i[1,0]])
            index_2 = np.array([i[0,1],i[1,1]])
            index_3 = np.array([i[0,2],i[1,2]])

            save_index_1.append(index_1)
            save_index_2.append(index_2)
            save_index_3.append(index_3)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if(abs(end-begin)>size_interval and begin<end):

                Number = 0
                T_prediction_sensor = []
                for sensor_data in GNSS_raw_data_corrected:
                    if sensor_data[0]<=end and sensor_data[0]>=begin:
                        T_prediction_sensor.append(sensor_data[0])
                        Index_sensor.append(Number)
                    Number = Number + 1

                #rate = 10  #Hz
                #T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))
                #print(T_prediction_sensor)
                T_prediction_init = torch.from_numpy(np.array(T_prediction_sensor))

                # Linear interpolation
                if(Mode == "L" or Mode == "All"):
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = prediction_u.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i)
                        P1_L, P2_L, P3_L = prediction_u.linear_prediction(i, 0, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_L)
                        Prediction_2.append(P2_L)
                        Prediction_3.append(P3_L)

        stop_time = time.time()
        print(stop_time - start_time)

        print("Interpolation finished !")

        if save:

            if(Mode == "L" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"

                if save:
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_1, path_out+trajectoire+ "_1.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_2, path_out+trajectoire+ "_2.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_3, path_out+trajectoire+ "_3.csv")

        print("Saved !")

# Trajectories predicted
Time = np.array(T_prediction)
P1_arr = np.array(Prediction_1)
P2_arr = np.array(Prediction_2)
P3_arr = np.array(Prediction_3)
Index_sensor = np.array(Index_sensor)

In [ ]:
# Plot of the interpolated trajectories
%matplotlib notebook
plt.figure()
plt.scatter(t1,tp1[0,:],s =5)                                               # Raw data from rosbag
plt.scatter(Time,P1_arr[:,0], s =2, linewidth=0.5, color= 'red')       # Interpolated points
plt.xlabel("Time [s]")
plt.ylabel("Axis chosen [m]")
plt.show()

In [ ]:
theodo_r = importlib.reload(theodo_r)
file_sensors = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/calibration_results.csv",'')
extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
error=[]
error = theodo_r.inter_prism_distance_error_mean(P1_arr, P2_arr, P3_arr, extrinsic_calibration_results[0:3])

In [ ]:
# Plot error according to timestamps
plt.figure()
plt.plot(Time, error)
plt.xlabel("Time [s]")
plt.ylabel("Error [mm]")
plt.yscale('log')
plt.show()

In [ ]:
# Outliers for results
P1_sort = []
P2_sort = []
P3_sort = []
Time_sort = []
Index_sensor_sort = []
threshold_inter_prims_error = 10    # In mm, need to check how to choose it
for i,j,k,l,m,n in zip(error,P1_arr,P2_arr,P3_arr,T_prediction,Index_sensor):
    if(i<threshold_inter_prims_error):
        P1_sort.append(j)
        P2_sort.append(k)
        P3_sort.append(l)
        Time_sort.append(m)
        Index_sensor_sort.append(n)
print("Number points input/output: ", len(P1_arr), len(P1_sort))

In [ ]:
# Load sensor positions
file_sensor_positions = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/sensor_positions.csv",'')
sensor_positions_list = theodo_u.read_extrinsic_calibration_results_file(file_sensor_positions)
GPS_data = True
Lidar_data = False
Gps_reference_chosen = 3    # 1: front, 2: back, 3: middle   # GNSS only for now

Sensors = []
P1_position_RTS = np.array(sensor_positions_list[0])
P2_position_RTS = np.array(sensor_positions_list[1])
P3_position_RTS = np.array(sensor_positions_list[2])
if GPS_data:
    Sensors.append(np.array(sensor_positions_list[3]))
    Sensors.append(np.array(sensor_positions_list[4]))
    Sensors.append(np.array(sensor_positions_list[5]))
if Lidar_data:
    Sensors.append(np.array(sensor_positions_list[6]))

P1_position_GNSS = P1_position_RTS - Sensors[Gps_reference_chosen-1]
P1_position_GNSS[3] = 1
P2_position_GNSS = P2_position_RTS - Sensors[Gps_reference_chosen-1]
P2_position_GNSS[3] = 1
P3_position_GNSS = P3_position_RTS - Sensors[Gps_reference_chosen-1]
P3_position_GNSS[3] = 1

P_sensor = np.array([P1_position_GNSS,
            P2_position_GNSS,
            P3_position_GNSS]).T

In [ ]:
# Doing a minimization between these not moving points, and the 3D prism coordinates
# Pose_GNSS is a list of each rigid transform founded
list_sensor_time = []
Pose_sensor = []
Prism_corrected = []
number = len(P1_sort)
for i in range(0,number):
    Q = np.array([P1_sort[i], P2_sort[i], P3_sort[i]]).T
    Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
    T = theodo_u.point_to_point_minimization(P_sensor, Q)
    Pose_sensor.append(T)
    prism_correct = T@P_sensor
    Prism_corrected.append(prism_correct)
    list_sensor_time.append(Time_sort[i])
Pose_sensor_arr = np.array(Pose_sensor)
Prism_corrected_arr = np.array(Prism_corrected)

In [ ]:
# Plot bird view of the results
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes()
begin = 0
end = -1
ax.scatter(Pose_sensor_arr[begin:end,0,3],Pose_sensor_arr[begin:end,1,3], color='black')
ax.scatter(Prism_corrected_arr[begin:end,0,0],Prism_corrected_arr[begin:end,1,0], color='red')
ax.scatter(Prism_corrected_arr[begin:end,0,1],Prism_corrected_arr[begin:end,1,1], color='green')
ax.scatter(Prism_corrected_arr[begin:end,0,2],Prism_corrected_arr[begin:end,1,2], color='blue')
ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")
ax.set_aspect('equal')
fig.tight_layout()

In [ ]:
# Plot X axis results
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes()
begin = 0
end = -1
ax.scatter(list_sensor_time[begin:end],Pose_sensor_arr[begin:end,0,3], color='black')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,0,0], color='red')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,0,1], color='green')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,0,2], color='blue')
ax.set_xlabel("Time [s]")
ax.set_ylabel("X [m]")
fig.tight_layout()

In [ ]:
# Plot Y axis results
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes()
begin = 0
end = -1
ax.scatter(list_sensor_time[begin:end],Pose_sensor_arr[begin:end,1,3], color='black')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,1,0], color='red')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,1,1], color='green')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,1,2], color='blue')
ax.set_xlabel("Time [s]")
ax.set_ylabel("Y [m]")
fig.tight_layout()

In [ ]:
# Plot Z axis results
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes()
begin = 0
end = -1
ax.scatter(list_sensor_time[begin:end],Pose_sensor_arr[begin:end,2,3], color='black')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,2,0], color='red')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,2,1], color='green')
ax.scatter(list_sensor_time[begin:end],Prism_corrected_arr[begin:end,2,2], color='blue')
ax.set_xlabel("Time [s]")
ax.set_ylabel("Z [m]")
fig.tight_layout()

In [ ]:
# Save result if GNSS TUM format
theodo_u = importlib.reload(theodo_u)
theodo_u.grountruth_GP_gps_convert_for_eval(list_sensor_time, Pose_sensor_arr, path + "ground_truth/gps_predicted_" + str(Gps_reference_chosen) +".txt")
theodo_u.grountruth_GNSS_sorted_convert_for_eval(Index_sensor_sort, path_GNSS_file, time_delay, path + "gps_data/gps_sorted_eval_" + str(Gps_reference_chosen) +".txt")